# Lesson 8. Pulling it all Together

Full challenge notebook
New datasets (2 or 3 max)
Ask a question
1. Pull i projection
2. Subset one by the other (within)
3. Walking distance (how many convenient stores are walking distance from each school)
Mini workflow from top to bottom
Or mini lesson on proximity analysis + then optional/homework challenge work flow


---
<div style="display:inline-block;vertical-align:middle;">
<a href="https://dlab.berkeley.edu/" target="_blank"><img src ="../assets/images/dlab_logo.png" width="75" align="left">
</a>
</div>

<div style="display:inline-block;vertical-align:middle;">
    <div style="font-size:larger">&nbsp;D-Lab @ University of California - Berkeley</div>
    <div>&nbsp;Team Geo<div>
</div>
        


